# Tutorial_yolov5_voc.ipynb

## 1. 라이브러리 설치 & 임포트

In [2]:
import torch
import torchvision
from torchvision import datasets
import os

print("PyTorch 버전 :", torch.__version__)
print("Torchvision 버전 :", torchvision.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("사용 중인 디바이스 :", device)

PyTorch 버전 : 2.6.0+cu118
Torchvision 버전 : 0.21.0+cu118
사용 중인 디바이스 : cuda


## 2. Pascal VOC 데이터셋 다운로드 (torchvision)

In [3]:
from torchvision.datasets import VOCDetection

# 로컬(혹은 Colab)에서 사용할 루트 경로 지정
# 예: 현재 디렉토리('.') 아래에 VOCdevkit 폴더가 생성됨
DATA_ROOT = "./"

# VOCDetection으로 train 데이터셋을 다운로드
voc_train = VOCDetection(
    root=DATA_ROOT,
    year="2007",
    image_set="train",  # trainval, test 등 가능
    download=True,       # 자동 다운로드
    transform=None,
    target_transform=None
)

print("VOC 2007 trainset 다운로드 완료!")
print("데이터 수:", len(voc_train))
print("저장 경로 구조 확인:", os.listdir(os.path.join(DATA_ROOT, "VOCdevkit")))


VOC 2007 trainset 다운로드 완료!
데이터 수: 2501
저장 경로 구조 확인: ['VOC2007']


## 3. VOC XML을 YOLO 텍스트로 변환하는 함수

In [5]:
import os
import xml.etree.ElementTree as ET
import glob
import random
import shutil
import numpy as np

# 경로 설정 - 수정된 VOCdevkit 경로
voc_root = "/userHome/userhome1/chaewoon/VOCdevkit/VOC2007"
voc_annotations = os.path.join(voc_root, "Annotations")
voc_images = os.path.join(voc_root, "JPEGImages")

# YOLO 데이터셋 저장 위치 - 같은 사용자 경로에 저장하여 권한 문제 해결
yolo_dataset = "/userHome/userhome1/chaewoon/yolo_voc_dataset"
os.makedirs(yolo_dataset, exist_ok=True)

# 클래스 목록
voc_classes = [
    "aeroplane", "bicycle", "bird", "boat", "bottle",
    "bus", "car", "cat", "chair", "cow",
    "diningtable", "dog", "horse", "motorbike", "person",
    "pottedplant", "sheep", "sofa", "train", "tvmonitor"
]

# VOC XML을 YOLO 텍스트로 변환하는 함수
def convert_voc_to_yolo(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    size = root.find("size")
    width = int(size.find("width").text)
    height = int(size.find("height").text)
    
    filename = root.find("filename").text
    base_name = os.path.splitext(filename)[0]
    
    yolo_lines = []
    
    for obj in root.findall("object"):
        # 클래스 이름 및 ID 가져오기
        class_name = obj.find("name").text
        if class_name not in voc_classes:
            continue
            
        class_id = voc_classes.index(class_name)
        
        # difficult 플래그 확인 (옵션)
        difficult = obj.find("difficult")
        if difficult is not None and int(difficult.text) == 1:
            continue
        
        # 바운딩 박스 좌표 가져오기
        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        
        # YOLO 형식으로 변환 (중심점 x, y, 너비, 높이) - 모두 0~1 사이 값
        x_center = ((xmin + xmax) / 2) / width
        y_center = ((ymin + ymax) / 2) / height
        box_width = (xmax - xmin) / width
        box_height = (ymax - ymin) / height
        
        # YOLO 형식으로 저장
        yolo_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
    
    # 변환된 라벨 저장
    if yolo_lines:  # 유효한 객체가 있는 경우에만 저장
        with open(os.path.join(output_dir, f"{base_name}.txt"), "w") as f:
            f.write("\n".join(yolo_lines))
        return True
    return False

## 4. 데이터셋 구성 (train/val 분할)

In [6]:
def setup_yolo_dataset():
    # 디렉토리 생성
    os.makedirs(os.path.join(yolo_dataset, "images", "train"), exist_ok=True)
    os.makedirs(os.path.join(yolo_dataset, "images", "val"), exist_ok=True)
    os.makedirs(os.path.join(yolo_dataset, "labels", "train"), exist_ok=True)
    os.makedirs(os.path.join(yolo_dataset, "labels", "val"), exist_ok=True)
    
    # YOLO 라벨 저장 디렉토리
    temp_labels_dir = os.path.join(yolo_dataset, "temp_labels")
    os.makedirs(temp_labels_dir, exist_ok=True)
    
    # 모든 XML 파일에 대한 라벨 변환
    xml_files = glob.glob(os.path.join(voc_annotations, "*.xml"))
    valid_images = []
    
    for xml_file in xml_files:
        base_name = os.path.splitext(os.path.basename(xml_file))[0]
        img_file = os.path.join(voc_images, f"{base_name}.jpg")
        
        # 이미지 파일이 존재하고 라벨 변환에 성공한 경우만 유효
        if os.path.exists(img_file) and convert_voc_to_yolo(xml_file, temp_labels_dir):
            valid_images.append(base_name)
    
    print(f"총 {len(valid_images)}개의 유효한 이미지와 라벨을 찾았습니다.")
    
    # train/val 분할 (80/20)
    random.shuffle(valid_images)
    split_idx = int(len(valid_images) * 0.8)
    train_images = valid_images[:split_idx]
    val_images = valid_images[split_idx:]
    
    print(f"학습용: {len(train_images)}개, 검증용: {len(val_images)}개")
    
    # 이미지와 라벨 파일 복사
    for img_set, subset in [(train_images, "train"), (val_images, "val")]:
        for img_name in img_set:
            # 이미지 복사
            src_img = os.path.join(voc_images, f"{img_name}.jpg")
            dst_img = os.path.join(yolo_dataset, "images", subset, f"{img_name}.jpg")
            shutil.copy(src_img, dst_img)
            
            # 라벨 복사
            src_label = os.path.join(temp_labels_dir, f"{img_name}.txt")
            dst_label = os.path.join(yolo_dataset, "labels", subset, f"{img_name}.txt")
            if os.path.exists(src_label):
                shutil.copy(src_label, dst_label)
    
    # 임시 라벨 디렉토리 삭제
    # shutil.rmtree(temp_labels_dir)
    
    return train_images, val_images


## 5. data.yaml 파일 생성

In [7]:
def create_data_yaml():
    # 경로 내 백슬래시 처리 (백슬래시를 슬래시로 변경)
    dataset_path = yolo_dataset.replace('\\', '/')
    
    yaml_content = """
# YOLOv5/v8 학습을 위한 데이터 설정
path: {0}
train: images/train  # train 이미지 디렉토리 경로 (path에 상대적)
val: images/val  # validation 이미지 디렉토리 경로 (path에 상대적)

# 클래스 정보
nc: {1}  # 클래스 수
names: {2}  # 클래스 이름
""".format(dataset_path, len(voc_classes), voc_classes)
    
    # yaml 파일 경로
    yaml_path = os.path.join(yolo_dataset, "data.yaml")
    
    try:
        # 파일 쓰기 시도
        with open(yaml_path, "w") as f:
            f.write(yaml_content)
    except PermissionError:
        # 권한 오류 발생 시 현재 디렉토리에 저장
        current_dir = os.getcwd()
        yaml_path = os.path.join(current_dir, "data.yaml")
        
        # data.yaml 내용 업데이트 (상대 경로 사용)
        yaml_content = yaml_content.replace(dataset_path, os.path.relpath(yolo_dataset, current_dir).replace('\\', '/'))
        
        with open(yaml_path, "w") as f:
            f.write(yaml_content)
        
        print(f"권한 오류로 인해 현재 디렉토리에 data.yaml을 저장했습니다: {yaml_path}")
    
    return yaml_path

# 전체 과정 실행
print("VOC 데이터셋을 YOLO 형식으로 변환 중...")
try:
    train_images, val_images = setup_yolo_dataset()
    yaml_path = create_data_yaml()
    print(f"변환 완료! data.yaml 파일: {yaml_path}")
except PermissionError as e:
    print(f"권한 오류 발생: {e}")
    print("홈 디렉토리를 사용하여 다시 시도합니다...")
    
    # 홈 디렉토리로 경로 변경
    import os.path
    home_dir = os.path.expanduser("~")
    yolo_dataset = os.path.join(home_dir, "yolo_voc_dataset")
    os.makedirs(yolo_dataset, exist_ok=True)
    
    # 다시 실행
    train_images, val_images = setup_yolo_dataset()
    yaml_path = create_data_yaml()
    print(f"변환 완료! data.yaml 파일: {yaml_path}")

VOC 데이터셋을 YOLO 형식으로 변환 중...
총 5011개의 유효한 이미지와 라벨을 찾았습니다.
학습용: 4008개, 검증용: 1003개
변환 완료! data.yaml 파일: /userHome/userhome1/chaewoon/yolo_voc_dataset/data.yaml


## 6. YOLOv5 학습 (Ultralytics) 

In [ ]:
from ultralytics import YOLO
import os
import torch

# 모델 로드
model = YOLO("yolov5s.pt")
print("YOLOv5 모델 로드 완료!")

# 학습 - GPU 사용
model.train(
    data=os.path.join(yolo_dataset, "data.yaml"),
    epochs=5,
    batch=4,
    imgsz=416,
    name="yolov5_voc_demo",
    device=0,
    workers=0    
)

사용 중인 디바이스: cuda
사용 가능한 GPU 수: 4
GPU 0: NVIDIA GeForce RTX 3090
GPU 1: NVIDIA GeForce RTX 3090
GPU 2: NVIDIA GeForce RTX 3090
GPU 3: NVIDIA GeForce RTX 3090
PRO TIP 💡 Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



100%|██████████| 17.7M/17.7M [00:01<00:00, 9.78MB/s]

YOLOv5 모델 로드 완료!
Ultralytics 8.3.91 🚀 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)


engine/trainer: task=detect, mode=train, model=yolov5s.pt, data=/userHome/userhome1/chaewoon/yolo_voc_dataset/data.yaml, epochs=5, time=None, patience=100, batch=4, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=0, project=None, name=yolov5_voc_demo2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, ker

100%|██████████| 755k/755k [00:00<00:00, 10.3MB/s]

...

                   all       1003       2621      0.755      0.708      0.759      0.557



5 epochs completed in 0.177 hours.
Optimizer stripped from runs/detect/yolov5_voc_demo2/weights/last.pt, 18.5MB
Optimizer stripped from runs/detect/yolov5_voc_demo2/weights/best.pt, 18.5MB

Validating runs/detect/yolov5_voc_demo2/weights/best.pt...
Ultralytics 8.3.91 🚀 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)
YOLOv5s summary (fused): 84 layers, 9,119,276 parameters, 0 gradients, 23.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 126/126 [00:08<00:00, 14.80it/s]


                   all       1003       2621      0.753       0.71      0.759      0.557
             aeroplane         44         57      0.892      0.874      0.943      0.754
               bicycle         46         60      0.765      0.883      0.884      0.642
                  bird         81        112      0.801      0.661      0.709      0.523
                  boat         32         57      0.557      0.579       0.53      0.323
                bottle         53         87      0.778       0.46      0.615      0.441
                   bus         41         53       0.71       0.74      0.823       0.64
                   car        147        259      0.885      0.784      0.896      0.676
                   cat         57         62      0.952      0.645      0.793      0.594
                 chair         89        155      0.731      0.555      0.657      0.449
                   cow         30         45       0.72      0.689      0.751      0.574
           diningtabl

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f341b6e2910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043

## 7. 검증(Validation) 및 추론 테스트 

In [ ]:
# 학습 결과로 나온 best.pt 모델을 불러와 val 진행
results = model.val()
print(results)

# 추론 테스트
import cv2
import matplotlib.pyplot as plt

# 테스트 이미지 경로 설정
# 직접 이미지 ID 지정
image_dir = "/userHome/userhome1/chaewoon/VOCdevkit/VOC2007/JPEGImages"

# 이미지 디렉토리에서 이미지 파일 목록 가져오기
import glob
image_files = glob.glob(os.path.join(image_dir, "*.jpg"))

if not image_files:
    print(f"경로에 이미지 파일이 없습니다: {image_dir}")
else:
    # 첫 번째 이미지 파일 사용
    sample_img_path = image_files[0]
    sample_img_id = os.path.splitext(os.path.basename(sample_img_path))[0]
    
    print(f"테스트할 이미지: {sample_img_path}")
    
    # 모델로 예측 수행
    preds = model.predict(source=sample_img_path, conf=0.25, save=True)
    output_img_path = preds[0].path
    
    # 시각화
    img_result = cv2.imread(output_img_path)
    img_result = cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8,8))
    plt.imshow(img_result)
    plt.axis("off")
    plt.title("YOLOv5 Inference on Pascal VOC")
    plt.show()

Ultralytics 8.3.91 🚀 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24253MiB)


val: Scanning /userHome/userhome1/chaewoon/yolo_voc_dataset/labels/val.cache... 1003 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1003/1003 [00:00<?, ?it/s]


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 251/251 [00:18<00:00, 13.35it/s]


                   all       1003       2621      0.754      0.707      0.759      0.558
             aeroplane         44         57      0.881       0.86      0.943      0.755
               bicycle         46         60      0.768      0.883      0.884      0.643
                  bird         81        112      0.802      0.661      0.709      0.521
                  boat         32         57      0.559      0.579       0.53      0.322
                bottle         53         87      0.781       0.46      0.615       0.44
                   bus         41         53      0.707      0.736      0.822      0.639
                   car        147        259       0.89      0.783      0.896      0.677
                   cat         57         62      0.952      0.642       0.79      0.594
                 chair         89        155      0.741      0.553      0.662      0.452
                   cow         30         45      0.719      0.684      0.749      0.579
           diningtabl

<Figure size 800x800 with 1 Axes>